# Relatório diário Covid-19
Script para acessar bases de dados, processar, calcular médias e elaborar gráficos

Maurício Garcia, janeiro 2021

## Bibliotecas / frameworks

In [ ]:
# Gerais
import pandas as pd
import numpy as np

# Manipulação de arquivos e pastas
import shutil
from pathlib import Path

# Datas
import datetime
from datetime import date
from datetime import datetime

# Elaboração dos gráficos
# https://matplotlib.org/api/pyplot_api.html
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
from matplotlib.font_manager import FontProperties

# Manipulação das imagens
# https://pillow.readthedocs.io/
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw 

In [ ]:
# Configurações
pd.set_option('display.max_columns', None) # mostra todas colunas

# 1. Dados do Brasil

Baixar o arquivo CSV do site do Ministério da Saúde  
https://covid.saude.gov.br/  
Clicar em "Arquivo CSV" na parte superior direita da tela

In [ ]:
# Alterar essas pastas conforme desejado
pasta_origem = '/Users/iMacMgar/Downloads/' # pasta onde será baixado
pasta_destino = '/Users/iMacMgar/Dropbox/DEV/_Python/covid/_dados/' # pasta onde será processado

In [ ]:
# Essa rotina prepara o nome do arquivo baixado com a data de hoje no seguinte padrão:
# HIST_PAINEL_COVIDBR_05jan2021.csv
# Esse é o padrão do nome do arquivo gerado pelo Ministério da Saúde

ano = date.today().year
ano = str(ano)

meses = ['jan', 'fev', 'mar', 'abr', 'mai', 'jun', 'jul', 'ago', 'set', 'out', 'nov', 'dez']
mes = date.today().month
mes = meses[mes-1]
    
dia = date.today().day
dia = '0' + str(dia)
dia = dia[-2:]

arquivo = 'HIST_PAINEL_COVIDBR_' + dia + mes + ano + '.csv'

In [ ]:
# Move o arquivo baixado para a pasta de trabalho
now = datetime.now().strftime("%Y%m%d%H%M%S")
if Path(pasta_origem + arquivo).is_file():
    if Path(pasta_destino + 'DadosMS.csv').is_file(): # Faz backup de arquivo se existir
        shutil.move(pasta_destino + 'DadosMS.csv', pasta_destino + 'DadosMS_' + now +'.csv')
    shutil.move(pasta_origem + arquivo, pasta_destino + 'DadosMS.csv')

In [ ]:
# Abre o arquivo baixado
df = pd.read_csv(pasta_destino + 'DadosMS.csv', delimiter=";")

In [ ]:
# Array com estados
ufs = ['AC', 'AL', 'AM', 'AP', 'BA', 
       'CE', 'DF', 'ES', 'GO', 'MA', 
       'MG', 'MS', 'MT', 'PA', 'PB', 
       'PE', 'PI', 'PR', 'RJ', 'RN', 
       'RO', 'RR', 'RS', 'SC', 'SE', 
       'SP', 'TO']

# População dos estados
ufspop = [881935, 3337357, 4144597, 845731, 14873064, 
          9132078, 3015268, 4018650, 7018354, 7075181, 
          21168791, 2778986, 3484466, 8602865, 4018127, 
          9557071, 3273227, 11433957, 17264943, 3506853, 
          1777225, 605761, 11377239, 7164788, 2298696, 
          45919049, 1572866]

## 1.1. Casos e óbitos nacionais

In [ ]:
# Filtra apenas dados do Brasil
df1 = df[df['regiao'] == 'Brasil'].copy()

In [ ]:
# Calcula médias móveis
df1['BRcasos'] = df1['casosNovos'].rolling(window=7).mean()
df1['BRobitos'] = df1['obitosNovos'].rolling(window=7).mean()

In [ ]:
# Exibe dados dos últimos 8 dias
df1[['data', 'casosNovos', 'casosAcumulado', 'BRcasos', 'obitosNovos', 'obitosAcumulado', 'BRobitos']].tail(8)

In [ ]:
# Valores acumulados
obitos_br_acumulado = df1['obitosAcumulado'].tail(1).values[0]
casos_br_acumulado = df1['casosAcumulado'].tail(1).values[0]

In [ ]:
# Seleciona colunas e troca nulos por zero
df1 = df1[['data', 'BRcasos', 'BRobitos']].fillna(0)

In [ ]:
# Tabelão final, será usado na elaboração dos gráficos
dffull = df1

## 1.2 Óbitos por estado

In [ ]:
# Filtra apenas dados dos estados
df1 = df[[ 'municipio', 'codmun', 'data', 'estado', 'obitosNovos']].copy()
df1 = df1[df1['municipio'].isnull()]
df1 = df1[df1['codmun'].isnull()]
df1 = df1[df1['estado'].notnull()]
df1 = df1.drop(columns=['municipio'])
df1 = df1.drop(columns=['codmun'])
df1 = df1.sort_values(by=['estado'])
df1 = df1.sort_values(by=['estado', 'data'])
df1 = df1.reset_index()
df1 = df1.drop(columns=['index'])

In [ ]:
# Salva óbitos acumulados no estado
SPest_obitos = df1[df1['estado'] == 'SP']['obitosNovos'].sum()
RJest_obitos = df1[df1['estado'] == 'RJ']['obitosNovos'].sum()

In [ ]:
# Faz o pivo de óbitos por estado
pt = pd.pivot_table(df1, 
    index=['data'], #row
    columns=['estado'], #columns
    values=['obitosNovos']
)

In [ ]:
# Ajusta colunas, deixando com apenas uma linha
pt.columns = pt.columns.get_level_values(1)

In [ ]:
# Calcula a média móvel por estado
pt1 = pt.copy()
for uf in ufs:
    pt1[uf] = pt1[uf].rolling(window=7).mean().fillna(0)

In [ ]:
# Tabela final
dffull = pd.merge(dffull, pt1, on=['data'])

In [ ]:
# Calcula e mostra a diferença nos dois últimos dias
pt1 = pt1.tail(2).transpose()
pt1['dif'] = pt1.iloc[:, 1] - pt1.iloc[:, 0]
pt1 = pt1.sort_values(by=['dif'])
pt1 = pt1.transpose()
pt1.tail(1).transpose()

In [ ]:
# Exibe óbitos de hoje (para verificar zerados e/ou excessos)
pt2 = pt.tail(1).transpose()
pt2.sort_values(by=pt2.columns[0])

## 1.3. Óbitos SP e RJ Cidade

In [ ]:
# Dados de SP
municipio = 'São Paulo'
estado = 'SP'
df1 = df[df['municipio']==municipio]
df1 = df1[df1['estado']==estado]
SPcid_obitos = df1['obitosAcumulado'].tail(1).values[0]
df1 = df1[['data', 'obitosNovos']]
df1['SPcid'] = df1['obitosNovos'].rolling(window=7).mean().fillna(0)
df1 = df1[['data', 'SPcid']]

In [ ]:
# Tabelão final, será usado na elaboração dos gráficos
dffull = pd.merge(dffull, df1, on=['data'])

In [ ]:
# Dados do RJ
municipio = 'Rio de Janeiro'
estado = 'RJ'
df1 = df[df['municipio']==municipio]
df1 = df1[df1['estado']==estado]
RJcid_obitos = df1['obitosAcumulado'].tail(1).values[0]
df1 = df1[['data', 'obitosNovos']]
df1['RJcid'] = df1['obitosNovos'].rolling(window=7).mean().fillna(0)
df1 = df1[['data', 'RJcid']]

In [ ]:
# Tabelão final, será usado na elaboração dos gráficos
dffull = pd.merge(dffull, df1, on=['data'])

In [ ]:
# Interior
dffull['SPint'] = dffull['SP'] - dffull['SPcid']
dffull['RJint'] = dffull['RJ'] - dffull['RJcid']

In [ ]:
# Exibe dados dos últimos 8 dias
dffull[['data', 'SPcid', 'SPint', 'RJcid', 'RJint']].tail(8)

In [ ]:
# Gera data no formato dd/mm
dffull['data_m'] = dffull['data'].str[-2:] + '/' + dffull['data'].str[5:7]

## 1.4. Hospitais SP

In [ ]:
# Esses dados são obtidos manualmente, diariamente (!!!) 
# Dados da Cidade de SP: https://www.prefeitura.sp.gov.br/cidade/secretarias/saude/vigilancia_em_saude/doencas_e_agravos/coronavirus/index.php?p=295572
# Dados do Estado de SP: https://www.saopaulo.sp.gov.br/spnoticias/orgaos-governamentais/secretaria-da-saude/
# Depois de acessados, devem ser lançados na planilha abaixo

dfHosp = pd.read_excel('_dados/HospitaisSP.xlsx')

In [ ]:
# Calcula média móvel
for c in dfHosp.columns:
    if (c != 'data'):
        dfHosp[c] = dfHosp[c].rolling(window=7).mean().fillna(0)

In [ ]:
# Data no formato dd/mm
dfHosp['data'] = dfHosp['data'].astype(str)
dfHosp['data_m'] = dfHosp['data'].str[-2:] + '/' + dfHosp['data'].str[5:7]

In [ ]:
# Exibe dados dos últimos 8 dias
dfHosp.tail(8)

# 2. Dados internacionais

In [ ]:
# Fonte: https://github.com/owid/covid-19-data/tree/master/public/data
# Não precisa baixar, o script acessa diretamente no site

url = 'https://github.com/owid/covid-19-data/raw/master/public/data/owid-covid-data.csv'
df2 = pd.read_csv(url)

In [ ]:
# Países selecionados. Se houver interesse em outros países, basta acrescentar no array abaixo
# Sempre usar o nome do país em inglês
paises = [
    
    'Sweden',
    'Spain',
    'United Kingdom',
    'France',
    'Italy',
    'Germany',
    'Switzerland',
    'Portugal',
    'Russia',
    
    'India',
    'Japan',
    'South Korea',
    'Israel',
    'Bahrain',
    'United Arab Emirates',
    
    'Australia',
    'New Zealand',
    
    'South Africa',
    'Egypt',
    'Morocco',
    'Tunisia',
    'Nigeria',
    'Kenya',
    
    'United States',
    'Canada',
    'Mexico',
    'Peru',
    'Chile',
    'Argentina',
    'Uruguay'
]

In [ ]:
# Tabelão final, será usado na elaboração dos gráficos
dffull2 = df2[df2['location'] == 'World'][['date']].copy()

In [ ]:
# Média móvel de óbitos
for pais in paises:
    df1 = df2[df2['location'] == pais][['date','new_deaths']].copy()
    df1[pais] = df1['new_deaths'].rolling(window=7).mean().fillna(0)
    df1 = df1[['date',pais]]
    dffull2 = pd.merge(dffull2, df1, on=['date'])   

In [ ]:
# Dados sobre vacinação
vac = df2[['date', 'location', 'total_vaccinations_per_hundred']] 
vac = vac[vac['total_vaccinations_per_hundred'] > 0 ]
vac = vac.drop_duplicates(subset=['location'], keep='last')
vac = vac.sort_values(by=['total_vaccinations_per_hundred'], ascending=True)

# 3. Gráficos
Até esse ponto, todos os dados foram baixados e estão praticamente prontos  
para serem exibidos nos gráficos, que serão construídos a seguir.  
- Para a elaboração dos gráficos é usada a biblioteca Matplotlib Pyplot (plt)  
https://matplotlib.org/api/pyplot_api.html  
  
  
- Para a manipulação das imagens é usada a biblioteca Pillow (PIL)  
https://pillow.readthedocs.io/

In [ ]:
# Função de uso geral, para criar a imagem base de todos os gráficos
def new_image(titulo, subtitulo):
    # imagem de fundo
    image = Image.new('RGB', (4500, 4500), (256, 256, 256))
    # título e subtítulo
    font = ImageFont.truetype('Arial Bold.ttf', 120)
    ImageDraw.Draw(image).text((150, 100), titulo, font=font, fill=(72, 63, 137))
    font = ImageFont.truetype('Arial.ttf', 80)
    ImageDraw.Draw(image).text((150, 250), subtitulo, font=font, fill=(72, 63, 137))
    # data
    data = date.today().strftime('%d/%m/%Y')
    font = ImageFont.truetype('Arial.ttf', 80)
    ImageDraw.Draw(image).text((4000, 150), data, font=font, fill=(72, 63, 137))
    # Maurício
    cubo = Image.open('_dados/cuboW.png')
    image.paste(cubo, (150,4350))
    font = ImageFont.truetype('Arial.ttf', 60)
    ImageDraw.Draw(image).text((250, 4360), 'Maurício Garcia', font=font, fill=(72, 63, 137))
    return image

## 3.1. Brasil geral

In [ ]:
# Função para gerar gráficos
def grafico_br(x, y, cor, arquivo, esp):
    
    fig, ax = plt.subplots(figsize=(55, 20)) # cria a figura, cada ponto = 90 pixels - com dpi=90 - vide abaixo
    
    plt.fill_between( x, y, color=cor, alpha=0.1) # preenche a área sob o gráfico
    ax.tick_params(axis='both', which='major', labelsize=50) # labelsize = tamanho da fonte dos eixos x e y
    plt.box(on=None) # remove a caixa ao redor do gráfico
    ax.xaxis.set_major_locator(plticker.MultipleLocator(base=30)) # define o espaçamento do eixo x (horizontal)
    ax.yaxis.set_major_locator(plticker.MultipleLocator(base=esp)) # define o espaçamento do eixo y (vertical)
    ax.yaxis.grid() # acrescenta grid - linhas horizontais
    
    plt.plot(x, y, linewidth=30, color=cor, marker='o', mfc='white', ms='18') #plota o gráfico
    
    plt.savefig('_imagens/' + arquivo, dpi=90) # salva o arquivo

    plt.close(fig) # fecha o gráfico
    
    # crop da imagem
    image = Image.open('_imagens/' + arquivo)
    image = image.crop((380, 210, 4350, 1650))
    image.save('_imagens/' + arquivo)
    

In [ ]:
# Gera gráfico de óbitos
x = dffull['data_m'].values
y = dffull['BRobitos'].values
grafico_br(x, y, 'blue', 'BrasilOb.png', 100)

In [ ]:
# Gera gráfico de casos
x = dffull['data_m'].values
y = dffull['BRcasos'].values
grafico_br(x, y, 'green', 'BrasilCa.png', 5000)

In [ ]:
# Calcula valores de óbitos
vmax_ob = int(dffull['BRobitos'].max())
vhoj_ob = dffull['BRobitos'].tail(1).values[0].astype('int')
vhmm_ob = vhoj_ob/209.5
vhmm_ob = vhmm_ob.round(1)

In [ ]:
# Calcula valores de casos
vmax_ca = int(dffull['BRcasos'].max())
vhoj_ca = dffull['BRcasos'].tail(1).values[0].astype('int')

In [ ]:
# Formatando
obitos_br_acumulado_f = f'{obitos_br_acumulado:,}'
obitos_br_acumulado_f = obitos_br_acumulado_f.replace(',', '.')
casos_br_acumulado_f = f'{casos_br_acumulado:,}'
casos_br_acumulado_f = casos_br_acumulado_f.replace(',', '.')
vmax_ob_f = f'{vmax_ob:,}'
vmax_ob_f = vmax_ob_f.replace(',', '.')
vhoj_ob_f = f'{vhoj_ob:,}'
vhoj_ob_f = vhoj_ob_f.replace(',', '.')
vhmm_ob_f = str(vhmm_ob)
vhmm_ob_f = vhmm_ob_f.replace('.', ',')
vmax_ca_f = f'{vmax_ca:,}'
vmax_ca_f = vmax_ca_f.replace(',', '.')
vhoj_ca_f = f'{vhoj_ca:,}'
vhoj_ca_f = vhoj_ca_f.replace(',', '.')

In [ ]:
# Cria a imagem de fundo
image = new_image('Óbitos e casos por Covid-19 no Brasil', 'Média móvel nos últimos 7 dias')

In [ ]:
# Cola gráficos de óbitos e casos
image2 = Image.open('_imagens/BrasilOb.png')
image.paste(image2, (200,800))
image2 = Image.open('_imagens/BrasilCa.png')
image.paste(image2, (200,2800))

In [ ]:
# Títulos
font = ImageFont.truetype('Arial Bold.ttf', 100)
ImageDraw.Draw(image).text((500, 600), 'Óbitos', font=font, fill=('blue'))
ImageDraw.Draw(image).text((500, 2600), 'Casos', font=font, fill=('green'))

In [ ]:
# Valores
font = ImageFont.truetype('Arial.ttf', 80)
ImageDraw.Draw(image).text((1700, 600), 'Pico: ' + vmax_ob_f, font=font, fill=('blue'))
ImageDraw.Draw(image).text((3700, 600), 'Hoje: ' + vhoj_ob_f, font=font, fill=('blue'))
ImageDraw.Draw(image).text((3560, 720), 'Hoje/MM: ' + vhmm_ob_f, font=font, fill=('red'))
ImageDraw.Draw(image).text((3100, 1980), 'Acumulado: ' + obitos_br_acumulado_f, font=font, fill=('blue'))
ImageDraw.Draw(image).text((1700, 2600), 'Pico: ' + vmax_ca_f, font=font, fill=('green'))
ImageDraw.Draw(image).text((3600, 2600), 'Hoje: ' + vhoj_ca_f, font=font, fill=('green'))
ImageDraw.Draw(image).text((3100, 4000), 'Acumulado: ' + casos_br_acumulado_f, font=font, fill=('green'))

In [ ]:
# Salva imagem
file_name = '1.BrasilGeral.png'
image.save('_imagens/' + file_name)
image.close()

## 3.2. Brasil estados

In [ ]:
# Função para gerar gráficos dos estados 
def grafico_estados(x, y, uf, mm):
    fig, ax = plt.subplots(figsize=(40, 20))
    plt.fill_between( x, y, color='blue', alpha=0.2)    
    plt.axis('off')

    plt.text(0.15, 0.6, uf, fontsize=200, color='darkslateblue', transform=fig.transFigure )
    mm = str(mm.round(1))
    mm = mm.replace('.', ',')
    plt.text(0.15, 0.45, mm, fontsize=150, color='red', transform=fig.transFigure )
    
    plt.plot(x, y, linewidth=20, color='darkslateblue')
    plt.savefig('_imagens/' + uf + '.png', dpi=35)
    plt.close(fig)
    
    # crop da imagem
    image = Image.open('_imagens/' + uf + '.png')
    image = image.crop((200, 95, 1225, 600))
    image.save('_imagens/' + uf + '.png')

In [ ]:
# Gera gráficos por estado
i = 0
for uf in ufs: 
    td = dffull[uf].tail(1).values[0]
    mm = td*1000000/ufspop[i]
    x = dffull['data'].values
    y = dffull[uf].values
    grafico_estados(x, y, uf, mm)
    i = i + 1

In [ ]:
# Array dos estados - define a sequência em que os gráficos serão exibidos - 4 por linha
ests = ['AC', 'AM', 'PA', 'AP', 
        'RO', 'RR', 'TO', 'MA', 
        'PI', 'CE', 'RN', 'PB', 
        'PE', 'AL', 'SE', 'BA', 
        'GO', 'DF', 'MT', 'MS', 
        'MG', 'ES', 'RJ', 'SP', 
        'PR', 'SC', 'RS']

In [ ]:
# Cria a imagem de fundo
image = new_image('Óbitos por Covid-19 nos estados brasileiros', 'Média móvel nos últimos 7 dias')

In [ ]:
r = 0
c = 1
for est in ests:
    v = 450 + r*550

    if (c == 1):
        h = 50
        c = 2
    elif (c == 2):
        h = 1175
        c = 3
    elif (c == 3):
        h = 2300
        c = 4
    elif (c == 4):
        h = 3425
        c = 1
        r = r + 1
    
    image2 = Image.open('_imagens/' + est + '.png')
    image.paste(image2, (h, v))

In [ ]:
# Rodapé com explicação
font = ImageFont.truetype('Arial.ttf', 50)
ImageDraw.Draw(image).text((2000, 4360), '(números em vermelho representam atual mortalidade por milhão por dia)', font=font, fill=(256, 0, 0))

In [ ]:
# Salva imagem
file_name = '2.BrasilEstados.png'
image.save('_imagens/' + file_name)
image.close()

## 3.3. SP-RJ

In [ ]:
# Função para gerar gráficos das cidades
def grafico_cidade(x, y, cor, st, esp):
    fig, ax = plt.subplots(figsize=(58, 30))
    
    plt.fill_between( x, y, color=cor, alpha=0.1) # preenche a área sob o gráfico
    ax.tick_params(axis='both', which='major', labelsize=80) # labelsize = tamanho da fonte dos eixos x e y
    plt.box(on=None) # remove a caixa ao redor do gráfico
    ax.xaxis.set_major_locator(plticker.MultipleLocator(base=30)) # define o espaçamento do eixo x (horizontal)
    ax.yaxis.set_major_locator(plticker.MultipleLocator(base=esp)) # define o espaçamento do eixo y (vertical)
    ax.yaxis.grid() # acrescenta grid - linhas horizontais
    
    #plt.plot(x, y, linewidth=30, color=cor, marker='o', mfc='white', ms='18') #plota o gráfico
    plt.plot(x, y, linewidth=30, color=cor) #plota o gráfico
    
    plt.savefig('_imagens/' + st + '.png', dpi=50)
    plt.close(fig)
    
    # crop da imagem
    image = Image.open('_imagens/' + st + '.png')
    image = image.crop((263, 180, 2545, 1373))
    image = image.resize((2050, 1046), Image.ANTIALIAS)
    image.save('_imagens/' + st + '.png')
    

In [ ]:
# Função para calcular os indicadores da cidade
def indicadores(st, pop, acu):
    # calcula
    vmax = int(dffull[st].max())
    vhoj = dffull[st].tail(1).values[0].astype('int')
    vmmm = vhoj * 1000000 / pop
    vacum = acu
    # formata
    vmax = f'{vmax:,}'
    vmax = vmax.replace(',', '.')
    vhoj = f'{vhoj:,}'
    vhoj = vhoj.replace(',', '.')
    vacum = f'{vacum:,}'
    vacum = vacum.replace(',', '.')
    vmmm = str(vmmm.round(1))
    vmmm = vmmm.replace('.', ',')
    return [vmax, vhoj, vmmm, vacum]


In [ ]:
# Gera gráficos

x = dffull['data_m'].values

st = 'SPcid'
y = dffull[st].values
grafico_cidade(x, y, 'blue', st, 10)

st = 'SPint'
y = dffull[st].values
grafico_cidade(x, y, 'green', st, 20)

st = 'RJcid'
y = dffull[st].values
grafico_cidade(x, y, 'blue', st, 15)

st = 'RJint'
y = dffull[st].values
grafico_cidade(x, y, 'green', st, 10)

In [ ]:
# População
SPcid_populacao = 12252023
SPest_populacao = 45919049
RJcid_populacao = 6718903
RJest_populacao = 17264943

In [ ]:
# Calcula e formata indicadores
SPcid = indicadores('SPcid', SPcid_populacao, SPcid_obitos)
RJcid = indicadores('RJcid', RJcid_populacao, RJcid_obitos)
SPint = indicadores('SPint', SPest_populacao - SPcid_populacao, SPest_obitos - SPcid_obitos)
RJint = indicadores('RJint', RJest_populacao - RJcid_populacao, RJest_obitos - RJcid_obitos)

In [ ]:
# Cria a imagem de fundo
image = new_image('Óbitos por Covid-19 em SP e RJ', 'Média móvel nos últimos 7 dias')

In [ ]:
# Cola gráficos
image2 = Image.open('_imagens/SPcid.png')
image.paste(image2, (200,1000))
image2 = Image.open('_imagens/SPint.png')
image.paste(image2, (2400,1000))
image2 = Image.open('_imagens/RJcid.png')
image.paste(image2, (200,2700))
image2 = Image.open('_imagens/RJint.png')
image.paste(image2, (2400,2700))

In [ ]:
# Títulos
font = ImageFont.truetype('Arial Bold.ttf', 80)
ImageDraw.Draw(image).text((200, 800), 'SP Capital', font=font, fill=('blue'))
ImageDraw.Draw(image).text((2400, 800), 'SP Interior', font=font, fill=('green'))
ImageDraw.Draw(image).text((200, 2500), 'RJ Capital', font=font, fill=('blue'))
ImageDraw.Draw(image).text((2400, 2500), 'RJ Interior', font=font, fill=('green'))

In [ ]:
# Indicadores
font = ImageFont.truetype('Arial.ttf', 60)

ImageDraw.Draw(image).text((1000, 800), 'Pico: ' + SPcid[0], font=font, fill=('blue'))
ImageDraw.Draw(image).text((1930, 800), 'Hoje: ' + SPcid[1], font=font, fill=('blue'))
ImageDraw.Draw(image).text((1800, 900), 'Hoje/MM: ' + SPcid[2], font=font, fill=('blue'))
ImageDraw.Draw(image).text((1600, 1850), 'Acumulado: ' + SPcid[3], font=font, fill=('blue'))

ImageDraw.Draw(image).text((3250, 800), 'Pico: ' + SPint[0], font=font, fill=('green'))
ImageDraw.Draw(image).text((4180, 800), 'Hoje: ' + SPint[1], font=font, fill=('green'))
ImageDraw.Draw(image).text((4050, 900), 'Hoje/MM: ' + SPint[2], font=font, fill=('green'))
ImageDraw.Draw(image).text((3850, 1850), 'Acumulado: ' + SPint[3], font=font, fill=('green'))

ImageDraw.Draw(image).text((1000, 2500), 'Pico: ' + RJcid[0], font=font, fill=('blue'))
ImageDraw.Draw(image).text((1930, 2500), 'Hoje: ' + RJcid[1], font=font, fill=('blue'))
ImageDraw.Draw(image).text((1800, 2600), 'Hoje/MM: ' + RJcid[2], font=font, fill=('blue'))
ImageDraw.Draw(image).text((1600, 3550), 'Acumulado: ' + RJcid[3], font=font, fill=('blue'))

ImageDraw.Draw(image).text((3250, 2500), 'Pico: ' + RJint[0], font=font, fill=('green'))
ImageDraw.Draw(image).text((4180, 2500), 'Hoje: ' + RJint[1], font=font, fill=('green'))
ImageDraw.Draw(image).text((4050, 2600), 'Hoje/MM: ' + RJint[2], font=font, fill=('green'))
ImageDraw.Draw(image).text((3850, 3550), 'Acumulado: ' + RJint[3], font=font, fill=('green'))


In [ ]:
# Salva imagem
file_name = '3.SPRJ.png'
image.save('_imagens/' + file_name)
image.close()

## 3.4. Hospitais

In [ ]:
# Função para gerar gráficos dos hospitais
def grafico_hospitais(x, y, cor, st, esp):
    fig, ax = plt.subplots(figsize=(58, 30))
    
    plt.fill_between( x, y, color=cor, alpha=0.1) # preenche a área sob o gráfico
    ax.tick_params(axis='both', which='major', labelsize=80) # labelsize = tamanho da fonte dos eixos x e y
    plt.box(on=None) # remove a caixa ao redor do gráfico
    ax.xaxis.set_major_locator(plticker.MultipleLocator(base=30)) # define o espaçamento do eixo x (horizontal)
    ax.yaxis.set_major_locator(plticker.MultipleLocator(base=esp)) # define o espaçamento do eixo y (vertical)
    ax.yaxis.grid() # acrescenta grid - linhas horizontais
    
    plt.plot(x, y, linewidth=30, color=cor) #plota o gráfico

    plt.savefig('_imagens/hosp_' + st + '.png', dpi=50)
    plt.close(fig)
    
    # crop da imagem
    image = Image.open('_imagens/hosp_' + st + '.png')
    image = image.crop((175, 175, 2620, 1390))
    image = image.resize((2050, 1046), Image.ANTIALIAS)
    image.save('_imagens/hosp_' + st + '.png')
    

In [ ]:
# remove linhas iniciais zeradas
dfHosp = dfHosp[dfHosp['UTI_ESP'] > 0]

In [ ]:
# Gera gráficos

x = dfHosp['data_m'].values

st = 'UTI_ESP'
y = dfHosp[st].values
grafico_hospitais(x, y, 'red', st, 1000)

st = 'RESP'
y = dfHosp[st].values
grafico_hospitais(x, y, 'red', st, 200)

st = 'SUSP'
y = dfHosp[st].values
grafico_hospitais(x, y, 'red', st, 75)

st = 'INTERN'
y = dfHosp[st].values
grafico_hospitais(x, y, 'red', st, 250)

st = 'UTI_CSP'
y = dfHosp[st].values
grafico_hospitais(x, y, 'red', st, 100)

st = 'VENTIL'
y = dfHosp[st].values
grafico_hospitais(x, y, 'red', st, 75)



In [ ]:
# Cria a imagem de fundo
image = new_image('Antendimentos e internações em hospitais públicos de SP', 'Média móvel nos últimos 7 dias - Dados da Cidade de SP (exceto o primeiro gráfico)')

In [ ]:
# Cola gráficos

st = 'UTI_ESP'
image2 = Image.open('_imagens/hosp_' + st + '.png')
image.paste(image2, (100, 600))

st = 'RESP'
image2 = Image.open('_imagens/hosp_' + st + '.png')
image.paste(image2, (2300, 600))

st = 'SUSP'
image2 = Image.open('_imagens/hosp_' + st + '.png')
image.paste(image2, (100, 1900))

st = 'INTERN'
image2 = Image.open('_imagens/hosp_' + st + '.png')
image.paste(image2, (2300, 1900))

st = 'UTI_CSP'
image2 = Image.open('_imagens/hosp_' + st + '.png')
image.paste(image2, (100, 3200))

st = 'VENTIL'
image2 = Image.open('_imagens/hosp_' + st + '.png')
image.paste(image2, (2300, 3200))

In [ ]:
# Títulos
font = ImageFont.truetype('Arial Bold.ttf', 80)
ImageDraw.Draw(image).text((100, 450), 'Internações em UTI no Estado', font=font, fill=('red'))
ImageDraw.Draw(image).text((2300, 450), 'Atendimentos de quadros respiratórios', font=font, fill=('red'))
ImageDraw.Draw(image).text((100, 1750), 'Atendimentos com suspeita de Covid-19', font=font, fill=('red'))
ImageDraw.Draw(image).text((2300, 1750), 'Pacientes totais internados', font=font, fill=('red'))
ImageDraw.Draw(image).text((100, 3050), 'Pacientes internados em UTI', font=font, fill=('red'))
ImageDraw.Draw(image).text((2300, 3050), 'Pacientes em ventilação mecânica', font=font, fill=('red'))

In [ ]:
# Salva imagem
file_name = '4.Hospitais.png'
image.save('_imagens/' + file_name)
image.close()

## 3.5. Países

In [ ]:
# Função para gerar gráficos dos países 
def grafico_paises(x, y, uf, mm):
    fig, ax = plt.subplots(figsize=(40, 20))
    plt.fill_between( x, y, color='green', alpha=0.2)    
    plt.axis('off')

    plt.text(0.15, 0.6, uf, fontsize=200, color='green', transform=fig.transFigure )
    mm = str(mm.round(1))
    mm = mm.replace('.', ',')
    plt.text(0.15, 0.45, mm, fontsize=150, color='red', transform=fig.transFigure )
    
    plt.plot(x, y, linewidth=20, color='green')
    plt.savefig('_imagens/' + uf + '.png', dpi=35)
    plt.close(fig)
    
    # crop da imagem
    image = Image.open('_imagens/' + uf + '.png')
    image = image.crop((200, 95, 1225, 600))
    image.save('_imagens/' + uf + '.png')

In [ ]:
# Array dos países - define a ordem em que serão exibidos os gráficos, 4 por linha
# Os países nessa listas precisam estar contidos no array "paises"
pss = ['Sweden', 'France', 'Spain', 'Italy', 
       'United Kingdom', 'Germany', 'Portugal', 'Switzerland', 
       'Russia', 'India', 'Israel', 'Bahrain', 
       'Japan', 'South Korea', 'Australia', 'New Zealand', 
       'South Africa', 'Egypt', 'Morocco', 'Tunisia',  
       'Nigeria', 'Kenya', 'United States', 'Canada',
       'Mexico', 'Chile', 'Argentina', 'Uruguay'
]

In [ ]:
# Gera gráficos por país
for pais in paises: 
    pop = df2[df2['location'] == pais].tail(1)['population'].values[0]
    td = dffull2[pais].tail(1).values[0]
    mm = td*1000000/pop
    x = dffull2['date'].values
    y = dffull2[pais].values
    grafico_paises(x, y, pais, mm)

In [ ]:
# Cria a imagem de fundo
image = new_image('Óbitos por Covid-19 em países selecionados', 'Média móvel nos últimos 7 dias')

In [ ]:
r = 0
c = 1
for ps in pss:
    v = 450 + r*550

    if (c == 1):
        h = 50
        c = 2
    elif (c == 2):
        h = 1175
        c = 3
    elif (c == 3):
        h = 2300
        c = 4
    elif (c == 4):
        h = 3425
        c = 1
        r = r + 1
    
    image2 = Image.open('_imagens/' + ps + '.png')
    image.paste(image2, (h, v))

In [ ]:
# Rodapé com explicação
font = ImageFont.truetype('Arial.ttf', 50)
ImageDraw.Draw(image).text((2000, 4360), '(números em vermelho representam atual mortalidade por milhão por dia)', font=font, fill=(256, 0, 0))

In [ ]:
# Salva imagem
file_name = '5.Paises.png'
image.save('_imagens/' + file_name)
image.close()

## 3.6. Vacinas

In [ ]:
# Função para criar as tabelas
def tabela(table, arquivo):

    # cria o objeto 
    fig, ax = plt.subplots()
    
    # prepara o array com o conteúdo e as cores da tabela
    cell_text = []
    cell_colours = []
    i = 0
    for index, row in table.iterrows():
        country = row['Country']
        rate = row['Rate']
        rate = str(rate) + '%'
        cell_text.append([country, rate])
        if (i == 0):
            cell_colours.append(['lightcyan', 'lightcyan']) # para alternar a cor das linhas
            i = 1
        elif (i == 1):
            cell_colours.append(['cyan', 'cyan']) # para alternar a cor das linhas
            i = 0
            
    # monta a tabela
    tb = plt.table(
        cellText=cell_text, 
        colLabels=['País', 'Taxa'], # título das colunas
        colColours=['indigo']*len(cell_text), # cor do fundo cabeçalho
        cellColours=cell_colours, # cores do fundo das células
        colWidths=[0.5, 0.2], # largura das colunas
        cellLoc='center', # centraliza textos em cada coluna
        loc='center' # elimina espaços ao redor da tabela
    )

    # remove as linhas do grid
    for cell in tb.properties()['child_artists']:
        cell.set_linewidth(0)
    
    # cor e bold nos textos do cabeçalho
    tb[(0,0)].get_text().set_color('w')
    tb[(0,1)].get_text().set_color('w')
    tb[(0,0)].set_text_props(fontproperties=FontProperties(weight='bold'))
    tb[(0,1)].set_text_props(fontproperties=FontProperties(weight='bold'))

    # outras configurações
    tb.auto_set_font_size(False) # desabilita o ajuste automático da fonte
    tb.set_fontsize(36) # tamanho da fonte
    tb.scale(4, 6) # ajuste da largura e da altura de cada célula
    plt.axis('off') # remove os eixos

    # salva imagem com a tabela
    plt.savefig('_imagens/' + arquivo, bbox_inches='tight', dpi=150)
    
    # fecha o objeto - impede sua exibição
    plt.close(fig)

In [ ]:
# Seleciona países mais adiantados
vac2 = vac[vac['location'] != 'World']
vac2 = vac2.tail(148)
vac2 = vac2.sort_values(by=['total_vaccinations_per_hundred'], ascending=False)
vac2 = vac2.rename(index=str, columns={'location': 'Country'})
vac2 = vac2.rename(index=str, columns={'total_vaccinations_per_hundred': 'Rate'})
vac2['Rate'] = vac2['Rate'].round(1)

In [ ]:
# Gera as tabelas
tabela(vac2.head(23), 'vacina1.png')
tabela(vac2.tail(23), 'vacina2.png')

In [ ]:
# Cria a imagem de fundo
image = new_image('Porcentagem da população vacinada contra Covid-19', 'Países mais adiantados')

In [ ]:
# Cola tabelas
image2 = Image.open('_imagens/vacina1.png')
image.paste(image2, (200,600))
image2 = Image.open('_imagens/vacina2.png')
image.paste(image2, (2300,600))

In [ ]:
# Salva imagem
file_name = '6.Vacinas.png'
image.save('_imagens/' + file_name)
image.close()